In [56]:
from smolagents import tool
from typing import Optional, List, Union
import pandas as pd
import networkx as nx
import itertools

def convert_entities_to_json(row):
    # Initialize a dictionary to hold the entities grouped by their type
    entities_list = row['entities']
    entity_dict = {}

    entity_dict['timestamp'] = row['created_at']
    entity_dict['raw_text'] = row['text']


    for entity in entities_list:
        label = entity['label']
        text = entity['text']

        if label not in entity_dict:
            entity_dict[label] = ""

        # Append the entity text to the list of its corresponding type
        if len(entity_dict[label]) == 0:
            entity_dict[label] += text
        else:
            entity_dict[label] += ", "+text

    # Convert the dictionary to a JSON string
    #json_output = json.dumps(entity_dict, indent=4)
    
    return entity_dict


dataset="dump.pkl"

df = pd.read_pickle(dataset)
df = df[df['entities'].apply(lambda x: len(x) > 0)]
df = pd.DataFrame(df.apply(convert_entities_to_json, axis=1).to_list()).fillna("")

@tool
def load_filter_dataframe(
    start_time: Optional[str] = None,
    end_time: Optional[str] = None,
    match_columns: Optional[object] = None,  # Explicitly Optional
    match_strings: Optional[object] = None   # Explicitly Optional
) -> pd.DataFrame:
    """
    Filters the DataFrame based on an optional time range and text matches.

    Args:
        start_time: The starting timestamp to filter data from (format: 'YYYY-MM-DD HH:MM:SS').
        end_time: The ending timestamp to filter data until (format: 'YYYY-MM-DD HH:MM:SS').
        match_columns: A list of column names to search for matches (nullable).
        match_strings: Strings to match in the specified columns (nullable).

    Returns:
        Filtered pandas DataFrame containing only rows that match all criteria.

    Example:
        filter_dataframe(
            start_time='2023-01-01 00:00:00',
            end_time='2023-01-31 23:59:59',
            match_columns=['text', 'title'],
            match_strings=[['ai', 'machine learning', 'artificial intelligence', 'neural']])
    """
    global df
    
    # Convert timestamps
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Apply time filtering if start_time and/or end_time are provided
    if start_time is not None or end_time is not None:
        if start_time is not None:
            try:
                start_time = pd.to_datetime(start_time)
            except Exception as e:
                raise ValueError(f"Invalid start_time format: {e}")
        
        if end_time is not None:
            try:
                end_time = pd.to_datetime(end_time)
            except Exception as e:
                raise ValueError(f"Invalid end_time format: {e}")

        # Apply the time range filter
        if start_time and end_time:
            df = df[(df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)]
        elif start_time:
            df = df[df['timestamp'] >= start_time]
        elif end_time:
            df = df[df['timestamp'] <= end_time]

    # Apply text matching if match_columns and match_strings are provided
    if match_columns and match_strings:
        # Ensure match_strings is a list of lists if it's not already structured that way
        if isinstance(match_strings[0], str):
            match_strings = [match_strings]

        for column, strings in zip(match_columns, match_strings):
            if column in df.columns:
                # Use any() to check if any of the specified strings are present (case-insensitive)
                condition = df[column].apply(
                    lambda x: any(
                        s.lower() in str(item).lower()
                        for s in strings
                        for item in (x if isinstance(x, list) else [x])
                    )
                )
                df = df[condition]
            else:
                raise KeyError(f"Column '{column}' not found in DataFrame.")
    
    return df


@tool
def df_to_network_graph(df: pd.DataFrame, exclude_cols: Optional[object] = ['timestamp', 'raw_text']) -> nx.Graph:
    """
    Transform a DataFrame into a networkx graph where all entities in each row
    are connected to each other.
    
    Args:
        df: The input DataFrame to be transformed into a network graph.
        exclude_cols: Column names to exclude from the graph. Default excludes ['timestamp', 'raw_text'].
    
    Returns:
        A networkx.Graph with edges between entities that appear in the same row.
    """
    # Create empty graph
    
    G = nx.Graph()
    
    # Initialize list to exclude certain columns
    if exclude_cols is None:
        exclude_cols = []
    
    # Process each row
    for _, row in df.iterrows():
        # Get non-null values that aren't in exclude_cols
        entities = [str(row[col]) for col in df.columns 
                  if col not in exclude_cols and pd.notna(row[col])]
        
        # Create edges between all pairs of entities in this row
        for entity1, entity2 in itertools.combinations(entities, 2):
            if G.has_edge(entity1, entity2):
                # If edge exists, increment weight
                G[entity1][entity2]['weight'] += 1
            else:
                # Create new edge with weight 1
                G.add_edge(entity1, entity2, weight=1)
    
    return G




@tool
def write_findings(content: str, file_path: Optional[str] = "findings.md", append: Optional[bool] = False) -> str:
    """
    Writes or appends content to a markdown findings file.
    
    Args:
        content: The content to write to the file
        file_path: Path to the findings.md file. Defaults to 'findings.md'
        append: If True, appends to the file; if False, overwrites the file
        
    Returns:
        A message indicating success or failure
    """
    try:
        mode = 'a' if append else 'w'
        with open(file_path, mode) as file:
            file.write(content)
        return f"Successfully {'appended to' if append else 'wrote'} {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"

@tool
def write_plan_file(content: str, file_path: Optional[str] = "plan.md", append: Optional[bool] = False) -> str:
    """
    Writes or appends content to a markdown plan file.
    
    Args:
        content: The content to write to the file
        file_path: Path to the plan.md file. Defaults to 'plan.md'
        append: If True, appends to the file; if False, overwrites the file
        
    Returns:
        A message indicating success or failure
    """
    try:
        mode = 'a' if append else 'w'
        with open(file_path, mode) as file:
            file.write(content)
        return f"Successfully {'appended to' if append else 'wrote'} {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"



In [57]:
from smolagents.prompts import CODE_SYSTEM_PROMPT

#write_plan_file(CODE_SYSTEM_PROMPT)

In [58]:
new_prompt = """
You are an expert research assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you need to create/refine your plan in a structured markdown way. Putting a check where you've explored with some notes under each section.

# Research Agent Progress Tracking

## Completed Milestones
- [x] Initial project conception and research objective formulation
- [x] Comprehensive literature review on entity extraction and data processing methods
- [x] Proof of concept for Named Entity Recognition (NER) extraction using the Twitter/X dataset
- [x] Preliminary development of an entity and relation extraction prototype

## Current Research Focus
### Data Acquisition and Processing
- [x] Integration and cleaning of the Twitter/X dataset
- [x] Implementation of GLiNER for initial entity extraction
- [ ] Refinement of the entity taxonomy based on recent literature insights
- [ ] Optimization of relation extraction methodologies through iterative testing

Create a plan if there is no plan, update the plan after every observation. 

Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using notional tools:
---
Task: "Generate an image of the oldest person in this document."

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis

## Current Focus
### Information Gathering
- [ ] Identify the oldest person in the document
- [ ] Generate an appropriate image description
- [ ] Create final image

I'll proceed step by step, first using `document_qa` to find the oldest person mentioned in the document.

Code:
```
answer = document_qa(document=document, question="Who is the oldest person mentioned?")
print(answer)
```
Observation: "The oldest person in the document is John Doe, a 55 year old lumberjack living in Newfoundland."

Thought:
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis
- [x] Identified the oldest person in the document (John Doe, 55)

## Current Focus
### Image Generation
- [ ] Generate an appropriate image description
- [ ] Create final image

Now that I've identified John Doe as the oldest person, I'll generate an image portraying him.

Code:
```
image = image_generator("A portrait of John Doe, a 55-year-old man living in Canada.")
final_answer(image)
```

---
Task: "What is the result of the following operation: 5 + 3 + 1294.678?"

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Parsed mathematical expression

## Current Focus
### Calculation
- [ ] Compute the result (5 + 3 + 1294.678)
- [ ] Return final answer

This is a straightforward calculation task. I'll use Python to compute the result.

Code:
```
result = 5 + 3 + 1294.678
final_answer(result)
```

---
Task:
"Answer the question in the variable `question` about the image stored in the variable `image`. The question is in French.
You have been provided with these additional arguments, that you can access using the keys as variables in your python code:
{'question': 'Quel est l'animal sur l'image?', 'image': 'path/to/image.jpg'}"

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis

## Current Focus
### Language Processing
- [ ] Translate the question from French to English
- [ ] Analyze the image with the translated question
- [ ] Format the final answer

I need to first translate the question and then use image analysis to answer it.

Code:
```
translated_question = translator(question=question, src_lang="French", tgt_lang="English")
print(f"The translated question is {translated_question}.")
```
Observation: "The translated question is 'What is the animal in the image?'"

Thought:
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis
- [x] Translated the question from French to English

## Current Focus
### Image Analysis
- [ ] Analyze the image with the translated question
- [ ] Format the final answer

Now I'll use the translated question to analyze the image content.

Code:
```
answer = image_qa(image=image, question=translated_question)
final_answer(f"The answer is {answer}")
```

---
Task:
In a 1979 interview, Stanislaus Ulam discusses with Martin Sherwin about other great physicists of his time, including Oppenheimer.
What does he say was the consequence of Einstein learning too much math on his creativity, in one word?

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis

## Current Focus
### Information Retrieval
- [ ] Find the 1979 Ulam-Sherwin interview
- [ ] Locate Einstein references in the interview
- [ ] Extract the specific comment about Einstein and mathematics
- [ ] Identify the one-word consequence

I'll begin by searching for this specific interview.

Code:
```
pages = search(query="1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein")
print(pages)
```
Observation:
No result found for query "1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein".

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis
- [x] Attempted specific search (unsuccessful)

## Current Focus
### Information Retrieval
- [ ] Try broader search terms for the interview
- [ ] Locate Einstein references in the interview
- [ ] Extract the specific comment about Einstein and mathematics
- [ ] Identify the one-word consequence

The query was too restrictive. I'll try a broader search.

Code:
```
pages = search(query="1979 interview Stanislaus Ulam")
print(pages)
```
Observation:
Found 6 pages:
[Stanislaus Ulam 1979 interview](https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/)... [Ulam discusses Manhattan Project](https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/)

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Initial task analysis
- [x] Attempted specific search (unsuccessful)
- [x] Found relevant pages with broader search terms

## Current Focus
### Content Analysis
- [ ] Read the interview content from found pages
- [ ] Locate Einstein references in the interview
- [ ] Extract the specific comment about Einstein and mathematics
- [ ] Identify the one-word consequence

I'll examine the content of these pages to find the relevant information.

Code:
```
for url in ["https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/", "https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/"]:
    whole_page = visit_webpage(url)
    print(whole_page)
    print("\n" + "="*80 + "\n")  # Print separator between pages
```
Observation:
Manhattan Project Locations:
Los Alamos, NM
Stanislaus Ulam was a Polish-American mathematician. He worked on the Manhattan Project at Los Alamos and later helped design the hydrogen bomb. In this interview, he discusses his work at
(truncated)

Thought: I now have the final answer: from the webpages visited, Stanislaus Ulam says of Einstein: "He learned too much mathematics and sort of diminished, it seems to me personally, it seems to me his purely physics creativity." Let's answer in one word.
Code:
```
final_answer("diminished")
```

---
Task: "Which city has the highest population: Guangzhou or Shanghai?"

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Identified cities to compare (Guangzhou and Shanghai)

## Current Focus
### Data Collection
- [ ] Find population data for Guangzhou
- [ ] Find population data for Shanghai
- [ ] Compare populations
- [ ] Determine city with higher population

I need population figures for both cities to make the comparison.

Code:
```
for city in ["Guangzhou", "Shanghai"]:
    print(f"Population {city}:", search(f"{city} population"))
```
Observation:
Population Guangzhou: ['Guangzhou has a population of 15 million inhabitants as of 2021.']
Population Shanghai: '26 million (2019)'

Thought: 
# Task Progress Tracking

## Completed Milestones
- [x] Identified cities to compare (Guangzhou and Shanghai)
- [x] Found population data for Guangzhou (15 million in 2021)
- [x] Found population data for Shanghai (26 million in 2019)
- [x] Compared populations

## Current Focus
### Conclusion
- [ ] Determine city with higher population

Based on the data collected, Shanghai has a higher population (26 million) than Guangzhou (15 million).

Code:
```
final_answer("Shanghai")
```

---
Task: "What is the current age of the pope, raised to the power 0.36?"

Thought: I will use the tool `wiki` to get the age of the pope, and confirm that with a web search.
Code:
```
pope_age_wiki = wiki(query="current pope age")
print("Pope age as per wikipedia:", pope_age_wiki)
pope_age_search = web_search(query="current pope age")
print("Pope age as per google search:", pope_age_search)
```
Observation:
Pope age: "The pope Francis is currently 88 years old."

Thought: I know that the pope is 88 years old. Let's compute the result using python code.
Code:
```
pope_current_age = 88 ** 0.36
final_answer(pope_current_age)
```

Above example were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools:

{{tool_descriptions}}

{{managed_agents_descriptions}}

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:
``````' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: {{authorized_imports}}
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.
11. Write your thoughts out using the write_plan_file at the begining of every code block.
12. Update the write_plan_file and write_findings as a second last step just before calling the finish tool.
13. Use [x] and [ ] formatting in the plan
Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.

"""

In [ ]:
modified_system_prompt = new_prompt

from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id="bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0") # Could use 'gpt-4o'
agent = CodeAgent(tools=[write_findings,write_plan_file,load_filter_dataframe,], model=model,system_prompt=modified_system_prompt, additional_authorized_imports=['pandas','openpyxl','xlrd','os','matplotlib','numpy','statsmodels','PIL','io','base64','sklearn'])

result = agent.run(
    "Could you explore the loaded data frame and tell me what you see?",
)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ Could you explore the loaded data frame and tell me what you see?                                                                                                                                    │
│                                                                                                                                                                                                      │
╰─ LiteLLMModel - bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  # Update the plan                                                                                                                                                                                     
  write_plan_file('''                                                                                                                                                                                   
  # Research Agent Progress Tracking                                                                                                                                                                    
                                                                                                                                                                                                        
  ## Current Focus                                                                                                                                                                                      
  ### Data Exploration                                                                                                                                                                                  
  - [x] Load the data frame                                                                                                                                                                             
  - [ ] Examine the data frame structure                                                                                                                                                                
  - [ ] Analyze the content and data types                                                                                                                                                              
  - [ ] Provide a summary of findings                                                                                                                                                                   
  ''')                                                                                                                                                                                                  
                                                                                                                                                                                                        
  # Load the data frame without any filters                                                                                                                                                             
  df = load_filter_dataframe()                                                                                                                                                                          
                                                                                                                                                                                                        
  # Display basic information about the data frame                                                                                                                                                      
  print("Data frame info:")                                                                                                                                                                             
  print(df.info())                                                                                                                                                                                      
                                                                                                                                                                                      

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255270 entries, 0 to 255269
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   timestamp           255270 non-null  datetime64[ns, UTC]
 1   raw_text            255270 non-null  object             
 2   Event               255270 non-null  object             
 3   Company             255270 non-null  object             
 4   Product             255270 non-null  object             
 5   Location            255270 non-null  object             
 6   Growth Trend        255270 non-null  object             
 7   Competitor          255270 non-null  object             
 8   Time                255270 non-null  object             
 9   Financial Metrics   255270 non-null  object             
 10  Neutral Sentiment   255270 non-null  object             
 11  Negative Sentiment  255270 non-null  object             
 12  Regulation      

Execution logs:
Data frame info:
None

First few rows of the data frame:
                  timestamp  \
0 2020-04-09 23:56:51+00:00   
1 2020-04-09 23:54:47+00:00   
2 2020-04-09 23:54:26+00:00   
3 2020-04-09 23:49:05+00:00   
4 2020-04-09 23:49:01+00:00   

                                            raw_text            Event  \
0  $UMRX bouncing. EXTREMELY OVERSOLD #Coronaviru...      Coronavirus   
1  This week's Expired Signals are now published ...  Expired Signals   
2  #tradeitforward MTI's Ask An Analyst with Chri...                    
3  RT @CheddarFlow: Some highly unusual orders we...                    
4  RT @LanceRoberts: $JPM has stopped making Smal...                    

          Company   Product Location Growth Trend Competitor Time  \
0             GME                                                   
1  TGT, MCD, AAPL                                                   
2        MTI, MTI                                                   
3     CheddarFlow  SPY, SLV                                         
4             JPM                                                   

  Financial Metrics Neutral Sentiment Negative Sentiment Regulation Country  \
0                                                                             
1                                                                             
2                                                                             
3                                                                             
4                                                                             

  Positive Sentiment Measures  
0                              
1                              
2                              
3                              
4                              

Summary statistics:
                                  timestamp  \
count                                255270   
unique                                  NaN   
top                                     NaN   
freq                                    NaN   
mean    2020-05-30 13:13:38.167594752+00:00   
min               2020-04-09 00:00:01+00:00   
25%     2020-04-28 09:45:56.249999872+00:00   
50%               2020-06-06 02:20:14+00:00   
75%     2020-06-26 01:18:39.750000128+00:00   
max               2020-07-16 23:59:03+00:00   

                                                 raw_text   Event Company  \
count                                              255270  255270  255270   
unique                                             223752   11153   75031   
top     RT @eWhispers: #earnings for the week \n\nhttp...                   
freq                                                  510  222586   94164   
mean                                                  NaN     NaN     NaN   
min                                                   NaN     NaN     NaN   
25%                                                   NaN     NaN     NaN   
50%                                                   NaN     NaN     NaN   
75%                                                   NaN     NaN     NaN   
max                                                   NaN     NaN     NaN   

       Product Location Growth Trend Competitor    Time Financial Metrics  \
count   255270   255270       255270     255270  255270            255270   
unique   35710     3860         9316      13756   16095             12003   
top                                                                         
freq    147895   243709       235643     225436  196256            216162   
mean       NaN      NaN          NaN        NaN     NaN               NaN   
min        NaN      NaN          NaN        NaN     NaN               NaN   
25%        NaN      NaN          NaN        NaN     NaN               NaN   
50%        NaN      NaN          NaN        NaN     NaN               NaN   
75%        NaN      NaN          NaN        NaN     NaN               NaN   
max        NaN      NaN          NaN        NaN     NaN       

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                                                                                                                   
  import matplotlib.pyplot as plt                                                                                                                                                                       
  from collections import Counter                                                                                                                                                                       
                                                                                                                                                                                                        
  # Load the data                                                                                                                                                                                       
  df = load_filter_dataframe()                                                                                                                                                                          
                                                                                                                                                                                                        
  # Convert timestamp to datetime                                                                                                                                                                       
  df['timestamp'] = pd.to_datetime(df['timestamp'])                                                                                                                                                     
                                                                                                                                                                                                        
  # Analyze temporal distribution                                                                                                                                                                       
  daily_counts = df.resample('D', on='timestamp').size()                                                                                                                                                
                                                                                                                                                                                                        
  # Plot temporal distribution                                                                                                                                                                          
  plt.figure(figsize=(12, 6))                                                                                                                                                                           
  daily_counts.plot()                                                                                                                                                                                   
  plt.title('Temporal Distribution of Data')                                                                                                                                                            
  plt.xlabel('Date')                                                                                                                                                                                    
  plt.ylabel('Number of Entries')                                                                                                                                                     

Execution logs:
Updated findings with temporal distribution analysis and key feature identification.

Out: None

[Step 2: Duration 22.49 seconds| Input tokens: 40,963 | Output tokens: 2,157]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━